In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras import Input, Model
from keras.applications.resnet50 import ResNet50
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import clone_model
import mlflow
from utils import plot_loss_acc

import tensorflow as tf
print("Num GPUs Available:", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

## Loading data

In [ ]:
x_train = np.load('../data/x_train.npy', allow_pickle=True)
x_test = np.load('../data/x_test.npy', allow_pickle=True)
y_train = np.load('../data/y_train.npy', allow_pickle=True)
y_test = np.load('../data/y_test.npy', allow_pickle=True)

input_shape = x_train.shape[1:]
out_size = y_train.shape[1]

## Transfer Learning

## Res-Net

In [ ]:
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=input_shape)

base_model.trainable = False

head_model = base_model.output
head_model = GlobalAveragePooling2D(name="out_pool")(head_model)
head_model = Dense(out_size, activation="softmax", name="out_layer")(head_model)

model = Model(base_model.input, head_model)

In [ ]:
with mlflow.start_run(run_name="resnet"):
    BATCH_SIZE = 64
    EPOCHS = 20
    VAL_SPLIT = 0.1

    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

    history = model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=VAL_SPLIT)

    loss, accuracy = model.evaluate(x_test, y_test)
    print(loss, accuracy)

    params = {"epochs": EPOCHS, "batch_size": BATCH_SIZE}
    metrics = {"test_loss": loss, "test_acc": accuracy}

    mlflow.log_params(params)
    mlflow.log_metrics(metrics)
    
    for idx in range(len(history.history["loss"])):
        mlflow.log_metrics({'train_loss': history.history["loss"][idx], 'train_acc': history.history["accuracy"][idx], 
                           'val_loss': history.history["val_loss"][idx], 'val_acc': history.history["val_accuracy"][idx]}, step=idx+1)

    mlflow.keras.log_model(model, "model")

In [ ]:
plot_loss_acc(history)

### Data Augumentation

In [ ]:
with mlflow.start_run(run_name="resnet-data-aug"):
    BATCH_SIZE = 64
    EPOCHS = 15
    VAL_SPLIT = 0.1
    TRAIN_BATCH = 64
    VAL_BATCH = 64

    datagen = ImageDataGenerator(
        rotation_range=20, horizontal_flip=True, vertical_flip=True, validation_split=VAL_SPLIT)

    train_datagen = datagen.flow(x_train, y_train, batch_size=TRAIN_BATCH, subset="training")
    val_datagen = datagen.flow(x_train, y_train, batch_size=VAL_BATCH, subset="validation")

    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

    train_steps = x_train.shape[0] * (1 - VAL_SPLIT) // TRAIN_BATCH
    val_steps = x_train.shape[0] * VAL_SPLIT // VAL_BATCH
    
    history = model.fit(train_datagen, steps_per_epoch=train_steps, epochs=EPOCHS, 
                        validation_data=val_datagen, validation_steps=val_steps)
    
    loss, accuracy = model.evaluate(x_test, y_test)
    print(loss, accuracy)
    
    params = {"epochs": EPOCHS, "train_batch": TRAIN_BATCH, "val_batch": VAL_BATCH, "dense1": np.nan, "dense2": np.nan}
    metrics = {"test_loss": loss, "test_acc": accuracy}
    
    mlflow.log_params(params)
    mlflow.log_metrics(metrics)
    
    for idx in range(len(history.history["loss"])):
        mlflow.log_metrics({'train_loss': history.history["loss"][idx], 'train_acc': history.history["accuracy"][idx], 
                           'val_loss': history.history["val_loss"][idx], 'val_acc': history.history["val_accuracy"][idx]}, step=idx+1)

    mlflow.keras.log_model(model, "model")

In [ ]:
plot_loss_acc(history)

## Fine-tuning

### Unfreezing last conv block

In [ ]:
LAYER_NAME = "conv5_block3_1_conv"

for i, layer in enumerate(model.layers):
    if layer.name == LAYER_NAME:
        idx_conv_block = i

for layer in model.layers[idx_conv_block:]:
    layer.trainable = True

In [ ]:
with mlflow.start_run(run_name="resnet-fine-tuning"):
    
    BATCH_SIZE = 64
    EPOCHS = 15
    VAL_SPLIT = 0.1
    LR = 1e-5
    
    optim = Adam(LR)
    model.compile(loss="categorical_crossentropy", optimizer=optim, metrics=["accuracy"])

    history = model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=VAL_SPLIT)

    loss, accuracy = model.evaluate(x_test, y_test)
    
    print(loss, accuracy)
    
    params = {"epochs": EPOCHS, "batch_size": BATCH_SIZE, "layer_name": LAYER_NAME, "dense1": np.nan, "dense2": np.nan}
    metrics = {"test_loss": loss, "test_acc": accuracy}
    
    mlflow.log_params(params)
    mlflow.log_metrics(metrics)
    
    for idx in range(len(history.history["loss"])):
        mlflow.log_metrics({'train_loss': history.history["loss"][idx], 'train_acc': history.history["accuracy"][idx], 
                           'val_loss': history.history["val_loss"][idx], 'val_acc': history.history["val_accuracy"][idx]}, step=idx+1)

    mlflow.keras.log_model(model, "model")

In [ ]:
plot_loss_acc(history)

In [ ]:
y_pred = model.predict(x_test)
y_pred_class = np.argmax(y_pred, axis=1)
y_test_class = np.argmax(y_test, axis=1)

print(classification_report(y_test_class, y_pred_class))

In [ ]:
cm = confusion_matrix(y_test_class, y_pred_class)
cm_df = pd.DataFrame(cm)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_df, annot=True, fmt="d", cmap="Blues")
plt.ylabel('True class')
plt.xlabel('Predicted class')
# plt.savefig("../images/confusion_matrix.png")

In [ ]:
# Save model and weights
# model_json = model.to_json()
# with open("../data/model2.json", "w") as f:
#     f.write(model_json)

# model.save_weights("../data/weights2.h5")